# 2. Task的用法

In [1]:
using System.Threading;
using System.Threading.Tasks;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
public static class TaskClass
{
    public static void TaskMethod(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");
    }

    public static void TaskReport(Task task)
    {
        Console.WriteLine($"任务完成，完成时候的状态为： IsCanceled={task.IsCanceled}  IsCompleted={task.IsCompleted}  IsFaulted={task.IsFaulted}");
    }
    
    public static int TaskMethodInt(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");

        return 1024;
    }
}

## 2.1. 创建任务

### 2.1.1 无返回值的方式

In [3]:
//方式1，创建一个Task对象
Task t1 = new Task(() => TaskClass.TaskMethod("t1", 3000));
t1.Start();
Task.WaitAll(t1);  //阻塞，等待线程结束才能继续

Console.WriteLine("t1 is done.");

Task [t1] is start... ...
Task [t1] is running on a thread id: 5, is thread pool thread: True
Task [t1] is stop... ...
t1 is done.


In [4]:
//方式2，使用Task.Run()静态方法
Task.Run(() => TaskClass.TaskMethod("t2", 3000));

Console.WriteLine("t2 is done.");  //直接执行，非阻塞

t2 is done.
Task [t2] is start... ...
Task [t2] is running on a thread id: 29, is thread pool thread: True



(2,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [5]:
Task t2 = Task.Run(() => TaskClass.TaskMethod("t2", 3000));
Task.WaitAll(t2);  //阻塞，等待线程结束才能继续

Console.WriteLine("t2 is done.");  //直接执行，非阻塞

Task [t2] is start... ...
Task [t2] is running on a thread id: 5, is thread pool thread: True
Task [t2] is stop... ...
t2 is done.


In [6]:
//方式3，使用Task.Factory.StartNew()静态方法
//Task.Run()在.NET 4.5中引用，而Task.Factory.StartNew()更早，简单理解就是Task.Factory.StartNew()支持更多的定制化操作，而Task.Run()使用了更少的定制化来更简单的操作。
/*下面二者等价
Task.Run(someAction);
Task.Factory.StartNew(someAction, 
                      CancellationToken.None, TaskCreationOptions.DenyChildAttach, TaskScheduler.Default);
*/

Task.Factory.StartNew(() => TaskClass.TaskMethod("t3", 3000));  //直接异步的方法 

Console.WriteLine("t3 is done.");  //直接执行，非阻塞

t3 is done.
Task [t3] is start... ...
Task [t3] is running on a thread id: 29, is thread pool thread: True



(9,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [7]:
Task t3 = Task.Factory.StartNew(() => TaskClass.TaskMethod("t3", 3000));
Task.WaitAll(t3);  //阻塞，等待线程结束才能继续

Console.WriteLine("t3 is done.");

Task [t3] is start... ...
Task [t3] is running on a thread id: 5, is thread pool thread: True
Task [t3] is stop... ...
t3 is done.


In [8]:
//综合案例
Task t1 = new Task(() => TaskClass.TaskMethod("t1", 3000));
Task t2 = new Task(() => TaskClass.TaskMethod("t2", 3000));
t2.Start();
t1.Start();
Task.WaitAll(t1, t2);  //阻塞

Task.Run(() => TaskClass.TaskMethod("t3", 3000));
Task.Factory.StartNew(() => TaskClass.TaskMethod("t4", 3000));
Task.Factory.StartNew(() => TaskClass.TaskMethod("t5", 3000), TaskCreationOptions.LongRunning);  //标记为长时间运行任务，则任务不会使用线程池，而在单独的线程中运行

Task [t1] is start... ...
Task [t2] is start... ...
Task [t2] is running on a thread id: 5, is thread pool thread: True
Task [t1] is running on a thread id: 7, is thread pool thread: True
Task [t2] is stop... ...
Task [t1] is stop... ...
Task [t4] is start... ...
Task [t4] is running on a thread id: 29, is thread pool thread: True
Task [t5] is start... ...
Task [t5] is running on a thread id: 20, is thread pool thread: False
Task [t3] is start... ...
Task [t3] is running on a thread id: 7, is thread pool thread: True



(8,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.

(9,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.

(10,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [9]:
//常规使用方式
Console.WriteLine("主线程执行业务处理... ...");
//创建任务
Task task = new Task(() =>
{
    Console.WriteLine("使用System.Threading.Tasks.Task执行异步操作... ...");
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine(i);
    }
});
//启动任务，并安排到当前任务队列线程中执行任务(System.Threading.Tasks.TaskScheduler)
task.Start();
Console.WriteLine("主线程执行其他处理... ...");
task.Wait();

主线程执行业务处理... ...
主线程执行其他处理... ...
使用System.Threading.Tasks.Task执行异步操作... ...
0
1
2
3
4
5
6
7
8
9


In [10]:
//async/await的实现方式:
public async static void AsyncFunction()
{
    await Task.Delay(1);
    Console.WriteLine("使用System.Threading.Tasks.Task执行异步操作... ...");
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine(string.Format("AsyncFunction:i={0}", i));
    }
}

Console.WriteLine("主线程执行业务处理... ...");
AsyncFunction();
Console.WriteLine("主线程执行其他处理... ...");
for (int i = 0; i < 10; i++)
{
    Console.WriteLine(string.Format("Main:i={0}", i));
}

主线程执行业务处理... ...
主线程执行其他处理... ...
Main:i=0
Main:i=1
Main:i=2
Main:i=3
Main:i=4
Main:i=5
Main:i=6
Main:i=7
Main:i=8
Main:i=9
使用System.Threading.Tasks.Task执行异步操作... ...
AsyncFunction:i=0
AsyncFunction:i=1
AsyncFunction:i=2
AsyncFunction:i=3
AsyncFunction:i=4
AsyncFunction:i=5
AsyncFunction:i=6
AsyncFunction:i=7
AsyncFunction:i=8
AsyncFunction:i=9


### 2.1.2 带返回值的方式

In [11]:
Task<int> task = new Task<int>(() => TaskClass.TaskMethodInt("t1", 2000));
task.Start();

//模拟另一个任务
for (int i = 3; i > 0; i--)
{
    Console.WriteLine($"==== 1.{i} ====");
    Thread.Sleep(1000);
}

int result = task.Result;
Console.WriteLine($"task result is: {result}.");

//模拟另一个任务
for (int i = 3; i > 0; i--)
{
    Console.WriteLine($"==== 2.{i} ====");
    Thread.Sleep(1000);
}

==== 1.3 ====
Task [t1] is start... ...
Task [t1] is running on a thread id: 18, is thread pool thread: True
==== 1.2 ====
==== 1.1 ====
Task [t1] is stop... ...
task result is: 1024.
==== 2.3 ====
==== 2.2 ====
==== 2.1 ====


In [12]:
Task<int> task = new Task<int>(() => TaskClass.TaskMethodInt("t2", 2000));
task.RunSynchronously();  //运行在主线程中

//模拟另一个任务
for (int i = 3; i > 0; i--)
{
    Console.WriteLine($"==== 1.{i} ====");
    Thread.Sleep(1000);
}

int result = task.Result;
Console.WriteLine($"task result is: {result}.");

//模拟另一个任务
for (int i = 3; i > 0; i--)
{
    Console.WriteLine($"==== 2.{i} ====");
    Thread.Sleep(1000);
}

Task [t2] is start... ...
Task [t2] is running on a thread id: 5, is thread pool thread: True
Task [t2] is stop... ...
==== 1.3 ====
==== 1.2 ====
==== 1.1 ====
task result is: 1024.
==== 2.3 ====
==== 2.2 ====
==== 2.1 ====


In [13]:
Task<int> task = new Task<int>(() => TaskClass.TaskMethodInt("t2", 2000));
Console.WriteLine($"task.Status is: {task.Status}");
task.Start();

while (!task.IsCompleted)
{
    Console.WriteLine($"task.Status is: {task.Status}");
    Thread.Sleep(TimeSpan.FromSeconds(0.5));
}

Console.WriteLine($"task.Status is: {task.Status}");
result = task.Result;
Console.WriteLine($"task result is: {result}.");

task.Status is: Created
task.Status is: WaitingToRun
Task [t2] is start... ...
Task [t2] is running on a thread id: 19, is thread pool thread: True
task.Status is: Running
task.Status is: Running
task.Status is: Running
Task [t2] is stop... ...
task.Status is: RanToCompletion
task result is: 1024.


In [14]:
//常规使用方式
//创建任务
Task<int> getSumTask = new Task<int>(() => GetSum());
//启动任务，并安排到当前任务队列线程中执行任务(System.Threading.Tasks.TaskScheduler)
getSumTask.Start();
Console.WriteLine("主线程执行其他处理... ...");
//等待任务的完成执行过程。
getSumTask.Wait();
//获得任务的执行结果
Console.WriteLine("任务执行结果：{0}", getSumTask.Result.ToString());

public static int GetSum()
{
    int sum = 0;
    Console.WriteLine("使用Task执行异步操作... ...");
    for (int i = 0; i < 100; i++)
        sum += i;

    return sum;
}

主线程执行其他处理... ...
使用Task执行异步操作... ...
任务执行结果：4950


In [15]:
//async/await的实现方式:
public async static Task<int> AsyncGetSum()
{
    await Task.Delay(1);
    int sum = 0;
    Console.WriteLine("使用Task执行异步操作.");
    for (int i = 0; i < 100; i++)
    {
        sum += i;
    }
    return sum;
}

var ret = AsyncGetSum();
Console.WriteLine("主线程执行其他处理... ...");
for (int i = 1; i <= 3; i++)
    Console.WriteLine("Call Main()");
int result = ret.Result;  //阻塞主线程
Console.WriteLine("任务执行结果：{0}", result);

主线程执行其他处理... ...
Call Main()
Call Main()
Call Main()
使用Task执行异步操作.
任务执行结果：4950
